In [ ]:
%pip install dtale

In [ ]:
import cuml
import cupy as cp
from cuml.cluster.hdbscan import HDBSCAN
from sentence_transformers import SentenceTransformer
import numpy as np
import polars as pl
import dtale

In [ ]:
df = pl.read_parquet()

In [ ]:
# Convert the embeddings to a CuPy array
embeddings_gpu = cp.asarray(df["embeddings"].to_numpy())

In [ ]:
# Reduce the embeddings dimensions
umap_model = cuml.UMAP(
    n_neighbors=15, n_components=100, min_dist=0.1, metric="euclidean"
)
reduced_data_gpu = umap_model.fit_transform(embeddings_gpu)

In [ ]:
clusterer = HDBSCAN(
    min_cluster_size=10,
    gen_min_span_tree=True,
    metric="euclidean",
    cluster_selection_epsilon=0.02,
)
cluster_labels = clusterer.fit_predict(
    reduced_data_gpu.astype(np.float64).get()
)

In [ ]:
df.with_columns(cluster_label=pl.Series(cluster_labels))

In [ ]:
dtale(df)